### 구글드라이브 연결
> - drive 폴더가 붙은 것을 확인한다.
> - drwx------ 4 root root 4.0K Mar 22 11:21 drive

In [1]:
# from google.colab import drive 
# drive.mount('/content/drive/')

### Install / Import 모듈 

In [2]:
# 코랩에 없는 라이브러리 설치해주기
# !pip install bayesian-optimization

In [3]:
import os
import pandas as pd                         # 데이터 분석 라이브러리
import numpy as np                          # 계산 라이브러리
import matplotlib.pyplot as plt             # * 그래프 이미지

from tqdm import tqdm                       # 진행바

from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.model_selection import KFold   # K-fold CV    
from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
from functools import partial               # 함수 변수 고정
import lightgbm as lgb                      # LightGBM 라이브러리
import warnings           
import random                       
warnings.filterwarnings("ignore")           # 경고 문구 미표시

In [4]:
# 데이터가 많아 5분 정도 걸립니다.  
# 전체경로 = /content/drive/My Drive/Colab Notebooks/competition/c03_starcraft_prediction/data_raw/train.csv
# /content/drive/My Drive/Colab Notebooks/
# dir_base = '/content/drive/My Drive/Colab Notebooks/competition/''

dir_base = '/home/yk/0325_Starcraft/competition/'

raw      = 'c03_starcraft_prediction/data_raw/'
remake   = 'c03_starcraft_prediction/data_remake/'
submit   = 'c03_starcraft_prediction/data_submit/'
                                                                # WAY-01 = LGBM
filename_submit = 'submission_0413_way02_15_75_final.csv'       # WAY-02 = XGBM

### 사용함수의 정의

In [5]:
os.chdir(dir_base)
from _assets.modules import *
from _assets.module_data_preps import *
from _assets.module_xgbm_model import *


# OS 화일 및 DF 정보조회를 위한 탐색 모듈


# GAME_ID 유니크 데이터 전처리를 위한 모듈 - baseline offer


# XGBM_CV 모델 - by SongDo_StudyGroup Code 참조



Using TensorFlow backend.


### Data 전처리 및 저장하기
- 전처리 된 x_train, y_train 내용을 확인하고, 저장한다.

In [7]:
# 화일위치 / list 확인하기 ... 사용함수
# show_ls('/content')          # drive 가 붙었는지 확인!
show_ls(dir_base + remake)      # 불러올 Data 화일이름 확인!

False
DIR_TARGET=/home/yk/0325_Starcraft/competition
/home/yk/0325_Starcraft/competition/c03_starcraft_prediction/data_remake/
----------------------------------------
  01. .ipynb_checkpoints
  02. accumujated_winning_rate_per_time.csv
  03. accumulated_winning_rate_per_time.csv
  04. df_new_rate04_correct.csv
  05. df_win_rate_every_10sec.csv
  06. df_win_rate_every_20sec.csv
  07. df_win_rate_every_40sec.csv
  08. df_xtest_remake.csv
  09. df_xtest_remake_final.csv
  10. df_xtrain_remake.csv
  11. df_xtrain_remake_final.csv
  12. df_ytrain_remake.csv


In [8]:
%%time
""" df_xtrain_remake_final.csv / ytrain 값은 변함없음 (동일하게 사용)
# 데이터 전처리 과정 - 약 7 ~ 10분 정도 걸립니다 ... 여기서 세션 리셋 됨!
# x_train, y_train = data_preparation(train, answer=True)           # [ 67,091,776 x 7 ] ... 70%
# show_infoDF_from(x_train) # *** DATA SHAPE = [ 38,872 x 27 ] ... [ game_id ... ]
# show_infoDF_from(y_train) # *** DATA SHAPE = [ 38,872 x 3 ] .. ['Unnamed: 0', 'game_id', 'winner']
"""

x_train = pd.read_csv(dir_base + remake + 'df_xtrain_remake_final.csv')  # to <class 'pandas.core.frame.DataFrame'>
ytrain = pd.read_csv(dir_base + remake + 'df_ytrain_remake.csv')  # to <class 'numpy.ndarray'>
y_train = ytrain['winner'].values                                 # array([1, 1, 0, ..., 0, 1, 0])

CPU times: user 105 ms, sys: 18 ms, total: 123 ms
Wall time: 1.01 s


In [9]:
pd.Series(y_train).value_counts()
# 1    19499
# 0    19373            total = 38,872  y_train = np.array 38,872

1    19499
0    19373
dtype: int64

In [10]:
# 제대로 작성이 됬는지 확인 다 끝났으면, 저장한다. 저장끝났으면 비활성화 처리


"""  # 저장했으면 비활성 처리
df_xtrain.to_csv(dir_base + remake + 'df_xtrain_remake.csv')
df_ytrain.to_csv(dir_base + remake + 'df_ytrain_remake.csv')
"""

"  # 저장했으면 비활성 처리\ndf_xtrain.to_csv(dir_base + remake + 'df_xtrain_remake.csv')\ndf_ytrain.to_csv(dir_base + remake + 'df_ytrain_remake.csv')\n"

# XGBM Classifier 모델 적용
 - 송도 스터디그룹에서 유창준님 - XGBM 모델 코드 공유받음!

In [11]:
%%time
# 모델 그 외 변수는 고정
var_fixed = partial(
        XGB_cv, 
        x_data=x_train, 
        y_data=y_train, 
        n_splits=5, 
        output='score'
    )

# 베이지안 최적화 범위 설정
XGBo = BayesianOptimization(
        var_fixed, 
        {
            'max_depth': (8, 512),          # ~ 500
            'learning_rate': (0.1, 0.6),    # 0.6xxx
            'subsample': (0.5, 0.9),        # 0.5 ~ 0.7
            'colsample_bytree': (0.5, 1),   # 0.5 ~ 0.8
            # 'colsample_bynode':(0, 1),
            # 'n_estimators' : (16, 1024),
            'reg_alpha' : (7, 20),          # L1 - 7 ~ 20
            'reg_lambda' : (8, 50),         # L2 - 8 ~ 45
            'max_delta_step' : (1, 25),     # 8.x
            'gamma' : (5, 20),              # 1 ~ 15
        },
        random_state = random.randrange(50000) # 유동 시드 1~50000
    )

# XGBo.maximize(init_points=15, n_iter=60)  # 처음 15회 랜덤값으로 score 계산 후 60회 최적화 = 4/04(토)
XGBo.maximize(init_points=15, n_iter=75)    # 처음 15회 랜덤값으로 score 계산 후 75회 최적화 = 4/13(월) = final값 사용!

|   iter    |  target   | colsam... |   gamma   | learni... | max_de... | max_depth | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.6403   |  0.7301   |  14.38    |  0.4553   |  3.416    |  331.0    |  18.93    |  20.7     |  0.5334   |
|  2        |  0.6411   |  0.6795   |  14.67    |  0.4095   |  14.06    |  176.0    |  19.77    |  38.87    |  0.7737   |
|  3        |  0.6481   |  0.8281   |  11.1     |  0.2774   |  21.39    |  394.3    |  11.6     |  41.66    |  0.7849   |
|  4        |  0.6443   |  0.5765   |  8.782    |  0.431    |  4.29     |  303.7    |  17.06    |  35.34    |  0.5115   |
|  5        |  0.6459   |  0.9162   |  14.43    |  0.2374   |  4.436    |  320.5    |  8.404    |  31.02    |  0.656    |
|  6        |  0.6406   |  0.6738   |  18.97    |  0.3549   |  8.076    |  373.6    |  12.36    |  42.24    |  0.5875   |
|  7        |  0.6467   

|  40       |  0.6467   |  0.5547   |  5.562    |  0.3535   |  21.94    |  334.2    |  9.92     |  26.18    |  0.7286   |
|  41       |  0.653    |  0.7093   |  6.434    |  0.1525   |  2.87     |  84.77    |  9.495    |  47.09    |  0.6999   |
|  42       |  0.653    |  0.9691   |  5.41     |  0.132    |  24.5     |  424.1    |  9.765    |  12.4     |  0.794    |
|  43       |  0.6483   |  0.564    |  5.313    |  0.2106   |  6.906    |  9.381    |  19.17    |  49.01    |  0.5609   |
|  44       |  0.6476   |  0.8964   |  7.544    |  0.3707   |  20.74    |  369.9    |  9.879    |  11.7     |  0.7626   |
|  45       |  0.651    |  0.8188   |  6.318    |  0.2631   |  1.386    |  99.82    |  13.11    |  49.36    |  0.5699   |
|  46       |  0.6488   |  0.6195   |  10.18    |  0.4581   |  1.823    |  87.19    |  10.44    |  45.11    |  0.8368   |
|  47       |  0.6527   |  0.9926   |  5.788    |  0.2037   |  8.676    |  72.82    |  7.165    |  49.92    |  0.609    |
|  48       |  0.6509   

|  81       |  0.6521   |  0.6493   |  6.577    |  0.1135   |  24.68    |  152.6    |  19.12    |  14.32    |  0.893    |
|  82       |  0.6548   |  0.785    |  5.09     |  0.1157   |  1.417    |  363.6    |  8.848    |  45.01    |  0.7928   |
|  83       |  0.6402   |  0.9056   |  6.855    |  0.5854   |  2.337    |  85.39    |  7.786    |  42.94    |  0.6418   |
|  84       |  0.6531   |  0.5465   |  5.065    |  0.1248   |  24.99    |  446.8    |  17.4     |  8.509    |  0.6967   |
|  85       |  0.6543   |  0.7394   |  5.252    |  0.1041   |  20.88    |  402.1    |  7.177    |  48.14    |  0.8462   |
|  86       |  0.655    |  0.8256   |  6.768    |  0.158    |  24.96    |  294.2    |  8.137    |  24.85    |  0.8648   |
|  87       |  0.6538   |  0.5345   |  5.969    |  0.1067   |  24.8     |  291.2    |  10.77    |  20.24    |  0.6925   |
|  88       |  0.6526   |  0.5602   |  6.086    |  0.1517   |  13.3     |  511.3    |  16.63    |  30.28    |  0.8523   |
|  89       |  0.6541   

### 3.0 모델 학습 및 검증
- Model Tuning & Evaluation

> 1. AUC가 가장 높은 하이퍼 파라미터를 사용해 최종 모델을 얻습니다.
> 1. 훈련 세트와 같은 방법으로 테스트 세트에서 Feature를 추출합니다.
> 1. 최종 모델을 사용해 예측을 수행합니다.
> 1. 예측 결과를 submission.csv로 저장합니다.

In [12]:
"""
# params = lgbBO.max['params']
# models = lgb_cv(
#         params['num_leaves'], 
#         params['learning_rate'], 
#         params['n_estimators'], 
#         params['subsample'], 
#         params['colsample_bytree'], 
#         params['reg_alpha'], 
#         params['reg_lambda'], 
#         x_data=x_train, 
#         y_data=y_train, 
#         n_splits=5, 
#         output='model',
#    )
"""

params = XGBo.max['params']
models = XGB_cv(
        params['max_depth'], 
        params['learning_rate'],
        params['subsample'], 
        params['colsample_bytree'],
        # params['colsample_bynode'], 
        params['reg_alpha'], 
        params['reg_lambda'],
        params['max_delta_step'],
        # params['n_estimators'],
        params['gamma'],
        x_data=x_train, 
        y_data=y_train, 
        n_splits=5, 
        output='model'
    )

# TEST를 풀기위해, 학습데이터로 변경 (data_preps)
 1. Train Data set = [ 67,091,776 x 7 ] ... 70%
  - game_id 별로 정리하면 [38,872 x 27]로 압축된다. 
  - 경기당, 평균 1,726개의 이벤트가 존재한다.(압축률 0.057939 %)
 > - x_train set = *** DATA SHAPE = [ 38,872 x 27 ] pandas.DataFrame
 > - y_train set = *** DATA SHAPE = [ 38,872 x 1 ]  np.array
 
 1. Test Data set  = [ 28,714,849 x 6 ] ... 30%
  - game_id 별로 정리하면 [16,787 x 27] 로 압축된다. 
  - 경기당 평균 1711개의 이벤트가 존재한다.(압축률 0.058461 %)
 > - x_test set = *** DATA SHAPE = [16,787 x 27] pandas.DataFrame 
 > - y_test set = *** DATA SHAPE = [16,787 x 1] np.array =  대회주관자가 가지고 있음
 
 1. Submission = y_test set 
  - submission = [16,787 x 1] np.array to [16,787 x 2 ] pandas.DataFrame --> df.csv

In [13]:
"""
# 테스트용 데이터를 전처리 해서 분석 준비 ... (시간소요) 미리 저장한 화일을 불러온다
# 저장했으면 비활성 처리
# x_test, _ = data_preparation(test, answer=False)                 # [ 28,714,849 x 6 ] ... 30%
# x_test.to_csv(dir_base + remake + 'df_xtest_remake.csv')
# x_test = pd.read_csv(dir_base + remake + 'df_xtest_remake.csv')  # [  16,787 x 27 ]
"""
x_test = pd.read_csv(dir_base + remake + 'df_xtest_remake_final.csv')    # *** DATA SHAPE = [ 16,787 x 27 ]


# 예측결과 Submission 화일 만들기

In [14]:
"""
# pd.read_csv('data/sample_submission.csv', index_col=0)
# sample_submission = pd.read_csv(dir_base + sub_base + "sample_submission.csv")
"""

preds = []

for model in models:
    pred = model.predict_proba(x_test)[:, 1]
    preds.append(pred)
    
pred = np.mean(preds, axis=0)

In [15]:
show_ls(dir_base + raw)

False
DIR_TARGET=/home/yk/0325_Starcraft/competition/c03_starcraft_prediction/data_remake
/home/yk/0325_Starcraft/competition/c03_starcraft_prediction/data_raw/
----------------------------------------
  01. sample_submission.csv
  02. test.csv
  03. train.csv


In [16]:
submission = pd.read_csv(dir_base + raw + "sample_submission.csv") 


submission['winner'] = pred
submission.to_csv(dir_base + submit + filename_submit, index=False)

# 결과 확인하기
- 3월 데이콘 대회 홈페이지 = https://bit.ly/39bqWVg
- 결과 제출하기 = https://dacon.io/competitions/official/235583/mysubmission/

In [17]:
df_1 = submission
df_1

,game_id,winner
0,38872,0.651619
1,38873,0.499383
2,38874,0.434777
3,38875,0.187188
4,38876,0.472653
...,...,...
16782,55654,0.609708
16783,55655,0.347164
16784,55656,0.701390
16785,55657,0.503110
